<a href="https://colab.research.google.com/github/nvnsudharsan/AORC_data_download/blob/main/AORC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install s3fs zarr

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.3/199.3 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.8/85.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.8/8.8 MB 51.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 113.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 2.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 4.0.0 requires fsspec[http]<=2025.3.0,>=2023.1.0, but you have fsspec 2025.9.0 which is incompatible.
gcsfs 2025.3.0 requires fsspec==2025.3.0, but you have fsspec 2025.9.0 which is

In [2]:
import xarray as xr
import fsspec
import numpy as np
import s3fs
import zarr

In [3]:
base_url = f's3://noaa-nws-aorc-v1-1-1km'

In [4]:
import dask
from dask.distributed import Client
client = Client()
client

INFO:distributed.http.proxy:To route to workers diagnostics web server please install jupyter-server-proxy: python -m pip install jupyter-server-proxy
INFO:distributed.scheduler:State start
INFO:distributed.scheduler:  Scheduler at:     tcp://127.0.0.1:38339
INFO:distributed.scheduler:  dashboard at:  http://127.0.0.1:8787/status
INFO:distributed.scheduler:Registering Worker plugin shuffle
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:43563'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:44407'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:42151'
INFO:distributed.nanny:        Start Nanny at: 'tcp://127.0.0.1:34109'
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:44093 name: 2
INFO:distributed.scheduler:Starting worker compute stream, tcp://127.0.0.1:44093
INFO:distributed.core:Starting established connection to tcp://127.0.0.1:46016
INFO:distributed.scheduler:Register worker addr: tcp://127.0.0.1:44125 name: 0
INFO:

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 50.99 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38339,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:44125,Total threads: 2
Dashboard: http://127.0.0.1:34435/status,Memory: 12.75 GiB
Nanny: tcp://127.0.0.1:43563,


In [7]:
dask.config.set(temporary_directory='.')


In [19]:
dataset_years = list(range(2020,2025))


In [20]:
s3_out = s3fs.S3FileSystem(anon=True)
fileset = [s3fs.S3Map(
            root=f"{base_url}/{dataset_year}.zarr", s3=s3_out, check=False
        ) for dataset_year in dataset_years]

In [21]:
fileset

In [22]:
ds_multi_year = xr.open_mfdataset(fileset, engine='zarr')

In [23]:
ds_multi_year

<xarray.Dataset> Size: 99TB
Dimensions:              (time: 43848, latitude: 4201, longitude: 8401)
Coordinates:
  * latitude             (latitude) float64 34kB 20.0 20.01 20.02 ... 54.99 55.0
  * longitude            (longitude) float64 67kB -130.0 -130.0 ... -60.01 -60.0
  * time                 (time) datetime64[ns] 351kB 2020-01-01 ... 2024-12-3...
Data variables:
    APCP_surface         (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    DLWRF_surface        (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    DSWRF_surface        (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    PRES_surface         (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    SPFH_2maboveground   (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    TMP_2maboveground    (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    UGRD_10maboveground  (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>
    VGRD_10maboveground  (time, latitude, longitude) float64 12TB dask.array<chunksize=(144, 128, 256), meta=np.ndarray>

In [24]:
var='APCP_surface'


In [25]:
ds_multi_year[var]


<xarray.DataArray 'APCP_surface' (time: 43848, latitude: 4201, longitude: 8401)> Size: 12TB
dask.array<concatenate, shape=(43848, 4201, 8401), dtype=float64, chunksize=(144, 128, 256), chunktype=numpy.ndarray>
Coordinates:
  * latitude   (latitude) float64 34kB 20.0 20.01 20.02 ... 54.98 54.99 55.0
  * longitude  (longitude) float64 67kB -130.0 -130.0 -130.0 ... -60.01 -60.0
  * time       (time) datetime64[ns] 351kB 2020-01-01 ... 2024-12-31T23:00:00
Attributes:
    AORC_Contact:  aorc.info@noaa.gov
    aorc_version:  v1.1
    crs:           EPSG:4326
    level:         surface
    long_name:     Total Precipitation
    short_name:    APCP_surface
    units:         kg/m^2

In [18]:
print(f'Variable size: {ds_multi_year[var].nbytes/1e12:.1f} TB')


Variable size: 2.5 TB


In [26]:
var = 'TMP_2maboveground'


In [29]:
temp = ds_multi_year[var]


In [35]:
aorc_temp_20240510 = temp.sel(
    time='2024-05-10',
    longitude=slice(-98.7451, -96.3281),
    latitude=slice(29.2816,31.3888)
)


In [36]:
aorc_temp_20240510.to_netcdf('aorc_temp_20240510.nc')